<a href="https://colab.research.google.com/github/Joovvhan/korean-stt/blob/master/kang/num.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
      
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [52]:
params = list(net.parameters())
print(len(params))
print(params[0].size())
input = Variable(torch.randn(1,1,32,32))
out=net(input)
print(out)
out.backward(torch.randn(1,10))

10
torch.Size([6, 1, 5, 5])
tensor([[ 0.0820, -0.0370,  0.0356,  0.0697, -0.0518, -0.1030, -0.1364, -0.0349,
          0.0734, -0.1808]], grad_fn=<AddmmBackward>)


In [68]:
output = net(input)
target = Variable(torch.randn(1,10))
print(target)
target = target.view(1,-1)
print(target)
print(output)
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

tensor([[-1.0862, -1.1552, -0.6722,  1.4441, -1.0009,  0.3391, -1.4604,  1.9223,
         -0.4900, -0.0085]])
tensor([[-1.0862, -1.1552, -0.6722,  1.4441, -1.0009,  0.3391, -1.4604,  1.9223,
         -0.4900, -0.0085]])
tensor([[ 0.0820, -0.0370,  0.0356,  0.0697, -0.0518, -0.1030, -0.1364, -0.0349,
          0.0734, -0.1808]], grad_fn=<AddmmBackward>)
tensor(1.2032, grad_fn=<MseLossBackward>)


In [72]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])

net.zero_grad()

print(net.conv1.bias.grad)
loss.backward()

print(net.conv1.bias.grad)

tensor([0., 0., 0., 0., 0., 0.])
tensor([-0.0021, -0.0169,  0.0094,  0.0126,  0.0016, -0.0091])
